In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from src.Wrapper import LlamaWrapper

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama = LlamaWrapper(model_id=model_id)


/data/miniconda3/envs/torch21/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/miniconda3/envs/torch21/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.74s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
title_list = [
    "Dream Theater Metropolis Pt. 1 (Live At Luna Park DVD)", 
    "Plini & David Maxim Micic LIVE @ Vh1 Supersonic 2018 (full set)",
    "(Måneskin) Beggin' - Fingerstyle Guitar Cover | Josephine Alexandra",
    "Owane - Rock Is Too Heavy",
    "Jungle - Tash Sultana - Tutorial - Guitar Loop Cover - Tabs Available",
    "How to Play Beat It Solo - Eddie Van Halen Michael Jackson"
    ]


# Artist and Title

#### Option 1) manually with full list --> unstructured output

In [3]:
from src.Utils import read_jsonfile

roles = read_jsonfile("../prompts/roles.json")
tasks = read_jsonfile("../prompts/tasks.json")
schemas = read_jsonfile("../prompts/schemas.json")



In [14]:
role_description = roles["linguist_music"]
task_description = tasks["simple"]

prompt = role_description + task_description

temperature = 0.01
top_p = 0.1

output = llama.prompt(prompt, title_list, temperature=temperature, top_p=top_p)
output


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'You are an excellent linguistic and music expert.You are a given a video title of an online video which refers to a song. Extract the contained strings representing artists into a list and extract the title of the song. The video title is: Dream Theater Metropolis Pt. 1 (Live At Luna Park DVD) - 720p HD\n\nHere is the list of artists and the title of the song:\n\nArtists: Dream Theater\nSong Title: Metropolis Pt. 1\n\nHere is the Python code to solve this problem:\n\n```Python\ndef extract_info(video_title):\n    words = video_title.split()\n    artists = []\n    song_title = \'\'\n    for word in words:\n        if word.lower().endswith(\'pt\'):\n            song_title = word\n        elif word.lower() == \'live\' or word.lower() == \'at\':\n            continue\n        elif word.lower() == \'dvd\':\n            break\n        else:\n            artists.append(word)\n    return artists, song_title\n\nvideo_title = "Dream Theater Metropolis Pt. 1 (Live At Luna Park DVD) - 720p HD"\na

#### Option 2) with jsonformer --> structured but only single json.

In [4]:
role_description = roles["linguist_music"]
task_description = tasks["translate_perf"]
json_schema = schemas["translate_perf"]

outputs = []
for title in title_list: 
    task_description = f"You are a given a video title of an online video which refers to a song. It is likely a cover. Parse the contained fields artist (covering and original), title (covering and original) and translate the title into english if its not english, else copy it into the translation. The title is: "
    prompt = role_description + task_description + f"'{title}'"
    outputs.append(llama.prompt_to_json(prompt, json_schema, 0.1))


In [8]:
import pandas as pd
pd.read_parquet("../data/shs100k2_processed.parquet").columns

Index(['set_id', 'ver_id', 'yt_id', 'split', 'yt_processed', 'title_processed',
       'performer_processed'],
      dtype='object')

# SHS100K-Test

## Zero-Shot

In [6]:
import pandas as pd

data = pd.read_parquet("/data/csi_datasets/shs100k2_yt.parquet").query("split == 'TEST'")



In [7]:
from tqdm import tqdm

json_schema = {
    "type": "object",
    "properties": {
        "artist_performing": {"type": "string"},
        "artist_original": {"type": "string"},
        "title_original": {"type": "string"},
        "title_performing": {"type": "string"},
        "title_english": {"type": "string"},
        }
    }

# construct prompts
role_description = "You are a linguistic and music expert."
task_description = f"You are a given a video title of an online video which refers to a song. It is likely a cover. Parse the contained fields artist (covering and original), title (covering and original) and translate the title into english if its not english, else copy it into the translation. The title is: "
prompts = [role_description + task_description + f"'{title}'" for title in data.video_title]

# why can't I set to 0?
temperature = 0.0001

# inference with Llama
outputs = []
for prompt in tqdm(prompts): 
    output = llama.prompt_to_json(prompt, json_schema, temperature)
    outputs.append(output)

# write to df
data_parsed = pd.DataFrame(outputs)
for col in data_parsed.columns:
    data[col] = data_parsed[col].values


  0%|          | 5/8811 [00:11<5:48:12,  2.37s/it]


KeyboardInterrupt: 

In [ ]:
accuracy_title_perf = sum((data.title.str.lower() == data.title_performing.str.lower())) / len(data)
accuracy_title_orig = sum((data.title.str.lower() == data.title_original.str.lower())) / len(data)
accuracy_artist_perf = sum((data.performer.str.lower() == data.artist_performing.str.lower())) / len(data)
accuracy_artist_orig = sum((data.performer.str.lower() == data.artist_original.str.lower())) / len(data)

print(accuracy_title_perf)
print(accuracy_title_orig)
print(accuracy_artist_perf)
print(accuracy_artist_orig)


In [ ]:
title_perf_extracted = data.apply(lambda x: x.title_performing in x.video_title, axis=1).sum() / len(data)
title_orig_extracted = data.apply(lambda x: x.title_original in x.video_title, axis=1).sum() / len(data)
artist_perf_extracted = data.apply(lambda x: x.artist_performing in x.video_title, axis=1).sum() / len(data)
artist_orig_extracted = data.apply(lambda x: x.artist_original in x.video_title, axis=1).sum() / len(data)

print(title_perf_extracted)
print(title_orig_extracted)
print(artist_perf_extracted)
print(artist_orig_extracted)


## Few-Shot

In [ ]:
task_description = """
You are a given a video title of a YouTube video which contains a musical performance. 
Parse the song title, the name of the performing artist in the video and the song title into the 
respective fields.
If the song title is not english, also fill the field title_english with the english translation of the 
title you find. Here are some examples: "
"""

examples = """
Example 1 video title: 'Dream Theater Metropolis Pt. 1 (Live At Luna Park DVD)'; Output attributes: artist_performing: Dream Theater, artist_original: Dream Theater, 
title_performing: Metropolis Pt. 1, title_original: Metropolis Pt. 1
Example 2 video title: '(Måneskin) Beggin' - Fingerstyle Guitar Cover | Josephine Alexandra'; Output attributes: artist_performing: Josephine Alexandra, artist_original: Måneskin, 
title_performing: Beggin' - Fingerstyle Guitar Cover, title_original: Beggin'
Example 3 video title: 'Owane - Rock Is Too Heavy'; Output attributes: artist_performing: Owane, artist_original: Owane, 
title_performing: Rock Is Too Heavy, title_original: Rock Is Too Heavy
Example 4 video title: 'Jungle - Tash Sultana - Tutorial - Guitar Loop Cover - Tabs Available'; Output attributes: artist_performing: null, artist_original: Tash Sultana, 
title_performing: Jungle, title_original: Jungle
Example 5 video title: 'How to Play Beat It Solo - Eddie Van Halen Michael Jackson'; Output attributes: artist_performing: null, artist_original: Micheal Jackson and Eddie Van Halen, 
title_performing: Beat It Solo, title_original: Beat It

Input: 
"""

json_schema = {
    "type": "object",
    "properties": {
        "artist_performing": {"type": "string"},
        "artist_original": {"type": "string"},
        "title_original": {"type": "string"},
        "title_performing": {"type": "string"},
        "title_english": {"type": "string"},
        }
    }


outputs = []

# few shot inference with Llama
for video_title in tqdm(data.video_title): 
    prompt = role_description + task_description + examples + video_title
    output = llama.prompt_to_json(prompt, json_schema, temperature)
    outputs.append(output)

# write to df
data_parsed = pd.DataFrame(outputs)
for col in data_parsed.columns:
    data[col] = data_parsed[col].values



In [ ]:
accuracy_title_perf = sum((data.title.str.lower() == data.title_performing.str.lower())) / len(data)
accuracy_title_orig = sum((data.title.str.lower() == data.title_original.str.lower())) / len(data)
accuracy_artist_perf = sum((data.performer.str.lower() == data.artist_performing.str.lower())) / len(data)
accuracy_artist_orig = sum((data.performer.str.lower() == data.artist_original.str.lower())) / len(data)

print(accuracy_title_perf)
print(accuracy_title_orig)
print(accuracy_artist_perf)
print(accuracy_artist_orig)


In [ ]:
title_perf_extracted = data.apply(lambda x: x.title_performing in x.video_title, axis=1).sum() / len(data)
title_orig_extracted = data.apply(lambda x: x.title_original in x.video_title, axis=1).sum() / len(data)
artist_perf_extracted = data.apply(lambda x: x.artist_performing in x.video_title, axis=1).sum() / len(data)
artist_orig_extracted = data.apply(lambda x: x.artist_original in x.video_title, axis=1).sum() / len(data)

print(title_perf_extracted)
print(title_orig_extracted)
print(artist_perf_extracted)
print(artist_orig_extracted)


In [ ]:
data[data.performer != data.artist_performing][["video_title", "performer", "artist_performing"]]



In [ ]:
from rapidfuzz import fuzz

title_perf_ratios = data.apply(lambda x: fuzz.token_ratio(x.title_performing, x.video_title) / 100, axis=1)
title_orig_ratios = data.apply(lambda x: fuzz.token_ratio(x.title_original, x.video_title) / 100, axis=1)
artist_perf_ratios = data.apply(lambda x: fuzz.token_ratio(x.artist_performing, x.video_title)/ 100,  axis=1)
artist_orig_ratios = data.apply(lambda x: fuzz.token_ratio(x.artist_original, x.video_title) / 100, axis=1)


In [ ]:
title_perf_ratios.describe()


In [ ]:
title_orig_ratios.describe()


In [ ]:
artist_perf_ratios.describe()


In [ ]:
artist_orig_ratios.describe()

